# Integration Exec Summary

High-level PASS/FAIL dashboard for SQL integration packs.

Use this notebook for quick contract health review.
Use `FLINK_DATA_TRACE_AND_INTEGRATION_TESTS.ipynb` for deep incident analysis and row-level tracing.


In [1]:
import os
from datetime import datetime, timedelta, timezone
from pathlib import Path
import sys
_nb_helper_loaded = False
for _candidate in [Path.cwd().resolve(), *Path.cwd().resolve().parents]:
    _nb_dir = _candidate / 'tests' / 'python' / 'notebooks'
    if (_nb_dir / 'notebook_shared.py').is_file():
        if str(_nb_dir) not in sys.path:
            sys.path.insert(0, str(_nb_dir))
        _nb_helper_loaded = True
        break
if not _nb_helper_loaded:
    raise FileNotFoundError('Could not locate tests/python/notebooks/notebook_shared.py from current working directory')


import pandas as pd
import clickhouse_connect
from IPython.display import Markdown, display

from notebook_shared import resolve_validation_window, run_assertion_file, resolve_repo_root, resolve_sql_dir, style_status_table

REPO_ROOT = resolve_repo_root(Path.cwd())
SQL_DIR = resolve_sql_dir(REPO_ROOT)

CH_HOST = os.getenv('CH_HOST', 'localhost')
CH_PORT = int(os.getenv('CH_PORT', '8123'))
CH_DATABASE = os.getenv('CH_DATABASE', 'livepeer_analytics')
CH_USER = os.getenv('CH_USER', 'analytics_user')
CH_PASSWORD = os.getenv('CH_PASSWORD', 'analytics_password')
CH_SECURE = os.getenv('CH_SECURE', '').lower() in {'1', 'true', 'yes'}

LOOKBACK_HOURS = int(os.getenv('LOOKBACK_HOURS', '24'))
SCENARIO_LOOKBACK_HOURS = int(os.getenv('SCENARIO_LOOKBACK_HOURS', '720'))

FROM_TS, TO_TS, MAIN_WINDOW_SOURCE = resolve_validation_window(REPO_ROOT, LOOKBACK_HOURS)
SCENARIO_FROM_TS, SCENARIO_TO_TS, SCENARIO_WINDOW_SOURCE = resolve_validation_window(REPO_ROOT, SCENARIO_LOOKBACK_HOURS)

params_main = {
    'from_ts': FROM_TS.strftime('%Y-%m-%d %H:%M:%S.%f')[:-3],
    'to_ts': TO_TS.strftime('%Y-%m-%d %H:%M:%S.%f')[:-3],
}
params_scenario = {
    'from_ts': SCENARIO_FROM_TS.strftime('%Y-%m-%d %H:%M:%S.%f')[:-3],
    'to_ts': SCENARIO_TO_TS.strftime('%Y-%m-%d %H:%M:%S.%f')[:-3],
}

client = clickhouse_connect.get_client(
    host=CH_HOST,
    port=CH_PORT,
    username=CH_USER,
    password=CH_PASSWORD,
    database=CH_DATABASE,
    secure=CH_SECURE,
)

def query_df(sql: str, params: dict) -> pd.DataFrame:
    r = client.query(sql, parameters=params)
    return pd.DataFrame(r.result_rows, columns=r.column_names)

print(f'Repo root: {REPO_ROOT}')
print(f'SQL dir: {SQL_DIR}')
print(f'DB target: {CH_HOST}:{CH_PORT}/{CH_DATABASE}')
print(f'Main window source: {MAIN_WINDOW_SOURCE}')
print(f'Main window UTC: {FROM_TS.isoformat()} -> {TO_TS.isoformat()}')
print(f'Scenario window source: {SCENARIO_WINDOW_SOURCE}')
print(f'Scenario window UTC: {SCENARIO_FROM_TS.isoformat()} -> {SCENARIO_TO_TS.isoformat()}')


Repo root: /home/julian/Documents/development/spe-work/livepeer-naap-analytics
SQL dir: /home/julian/Documents/development/spe-work/livepeer-naap-analytics/tests/integration/sql
DB target: localhost:8123/livepeer_analytics
Main window source: manifest /home/julian/Documents/development/spe-work/livepeer-naap-analytics/tests/integration/fixtures/manifest.json
Main window UTC: 2026-02-25T16:08:39.871000+00:00 -> 2026-02-25T21:43:55.262000+00:00
Scenario window source: manifest /home/julian/Documents/development/spe-work/livepeer-naap-analytics/tests/integration/fixtures/manifest.json
Scenario window UTC: 2026-02-25T16:08:39.871000+00:00 -> 2026-02-25T21:43:55.262000+00:00


In [2]:
suite_specs = [
    ('Raw -> Typed', SQL_DIR / 'assertions_raw_typed.sql', params_main),
    ('Pipeline Contract', SQL_DIR / 'assertions_pipeline.sql', params_main),
    ('API Readiness', SQL_DIR / 'assertions_api_readiness.sql', params_main),
    ('Scenario Candidates', SQL_DIR / 'assertions_scenario_candidates.sql', params_scenario),
]

suite_results = {}
summary_rows = []

for suite_name, sql_file, suite_params in suite_specs:
    df = run_assertion_file(sql_file, suite_params, query_df)
    suite_results[suite_name] = df
    failures = int((df['status'] == 'FAIL').sum()) if (not df.empty and 'status' in df.columns) else 0
    summary_rows.append({
        'suite': suite_name,
        'tests': int(len(df)),
        'failures': failures,
        'status': 'PASS' if failures == 0 else 'FAIL',
        'window_utc': f"{suite_params['from_ts']} -> {suite_params['to_ts']}",
    })

summary_df = pd.DataFrame(summary_rows)
display(Markdown('## Executive Verdict'))
display(style_status_table(summary_df))


## Executive Verdict

,suite,tests,failures,status,window_utc
0,Raw -> Typed,5,0,PASS,2026-02-25 16:08:39.871 -> 2026-02-25 21:43:55.262
1,Pipeline Contract,30,0,PASS,2026-02-25 16:08:39.871 -> 2026-02-25 21:43:55.262
2,API Readiness,14,0,PASS,2026-02-25 16:08:39.871 -> 2026-02-25 21:43:55.262
3,Scenario Candidates,5,0,PASS,2026-02-25 16:08:39.871 -> 2026-02-25 21:43:55.262


In [3]:
display(Markdown('## Suite Overviews'))

for suite_name, _, _ in suite_specs:
    df = suite_results[suite_name]
    display(Markdown(f'### {suite_name}'))
    if df.empty:
        print('No tests loaded.')
        continue
    display(style_status_table(df[['test_name', 'status', 'failed_rows']]))


## Suite Overviews

### Raw -> Typed

,test_name,status,failed_rows
0,raw_typed_accepted_estimate_nonnegative,PASS,0.000000
1,raw_typed_no_dlq_or_quarantine_for_core_types,PASS,0.000000
2,raw_typed_core_1to1_parity,PASS,0.000000
3,raw_typed_network_capabilities_expected_in_window,PASS,0.000000
4,raw_typed_network_capabilities_fanout_guard,PASS,0.000000


### Pipeline Contract

,test_name,status,failed_rows
0,raw_events_present,PASS,0
1,capability_dimension_mvs_present,PASS,0
2,capability_dimensions_projecting,PASS,0
3,session_fact_present,PASS,0
4,network_capabilities_raw_and_typed_present,PASS,0
5,status_raw_to_silver_projection,PASS,0
6,trace_raw_to_silver_projection,PASS,0
7,ingest_raw_to_silver_projection,PASS,0
8,session_final_uniqueness,PASS,0
9,workflow_session_has_identifier,PASS,0


### API Readiness

,test_name,status,failed_rows
0,api_views_present,PASS,0
1,gpu_metrics_keys_not_null,PASS,0
2,gpu_metrics_rollup_fields_consistent,PASS,0
3,gpu_metrics_latency_fields_nonnegative,PASS,0
4,gpu_metrics_startup_seconds_matches_ms,PASS,0
5,network_demand_hourly_grain,PASS,0
6,network_demand_by_gpu_hourly_grain,PASS,0
7,network_demand_by_gpu_required_columns_present,PASS,0
8,network_demand_by_gpu_capacity_fields_nonnegative,PASS,0
9,network_demand_additive_fields_nonnegative,PASS,0


### Scenario Candidates

,test_name,status,failed_rows
0,scenario_1_clean_success_no_swap_fps_gt_12_exists,PASS,0
1,scenario_2_no_orchestrator_then_closed_exists,PASS,0
2,scenario_3_success_with_swap_exists,PASS,0
3,scenario_4_success_with_param_updates_exists,PASS,0
4,scenario_5_out_of_category_baseline_exists,PASS,0


In [4]:
display(Markdown('## Failure Details'))

for suite_name, _, _ in suite_specs:
    df = suite_results[suite_name]
    if df.empty or 'status' not in df.columns:
        continue
    failed = df[df['status'] == 'FAIL']
    if failed.empty:
        continue
    display(Markdown(f'### {suite_name}: Failed Tests'))
    display(style_status_table(failed))


## Failure Details